In [1]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import gensim
import pyLDAvis
import pyLDAvis.gensim


project_path = Path('..')

In [2]:
df = pd.read_csv('../data/pubmed/Anti-Saccade Task.csv')
df['abstract'].fillna(df['title'], inplace=True)

tqdm.pandas(desc='preprocessing abstracts')
df['preprocessed_abstract'] = df['abstract'].progress_apply(lambda abstract: gensim.parsing.preprocess_string(abstract))



/Users/morteza/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
preprocessing abstracts: 100%|██████████| 90/90 [00:00<00:00, 753.25it/s]


In [3]:
docs = df['preprocessed_abstract'].to_list()
words = gensim.corpora.Dictionary(docs)
corpus = [words.doc2bow(doc) for doc in docs]

NUM_TOPICS = 10

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=words,
                                            num_topics=NUM_TOPICS, 
                                            update_every=1,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
# lda_model.print_topics(num_words = 10)

print('Perplexity:', lda_model.log_perplexity(corpus))

coherence_score_lda = gensim.models.CoherenceModel(model=lda_model, texts=docs, dictionary=words, coherence='c_v')
coherence_score = coherence_score_lda.get_coherence()

print('\nCoherence Score:', coherence_score)

/Users/morteza/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Perplexity: -6.571966520602653

Coherence Score: 0.3685759313778553


In [5]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, words, mds='tsne')

/Users/morteza/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5      32.420952   8.115230       1        1  17.576976
0       0.648955  71.707100       2        1  16.375661
8      71.019684   9.971006       3        1  14.373974
2       3.785981  33.185135       4        1  13.481331
6     -42.666500  -9.222070       5        1  11.138092
1      -3.621689 -43.526516       6        1   8.737423
4      -3.632402  -4.171708       7        1   7.364812
3     -36.584400  35.804413       8        1   6.910506
9      44.391380  50.615383       9        1   3.600971
7      40.235809 -31.591665      10        1   0.440255, topic_info=          Term        Freq       Total Category  logprob  loglift
172         ey  107.000000  107.000000  Default  30.0000  30.0000
502        cue   60.000000   60.000000  Default  29.0000  29.0000
67        task  271.000000  271.000000  Default  28.0000  28.0000
188   movement   93.000000   93.000000  Default  27.0000  27.0000
1113       asd   23.000000   23.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
202      pupil    0.537336   12.961434  Topic10  -4.5891   2.2425
94      demand    0.245693    5.436810  Topic10  -5.3717   2.3287
643      human    0.413429   16.701594  Topic10  -4.8513   1.7268
425    subject    0.333485   63.450168  Topic10  -5.0662   0.1772
693      induc    0.254098    7.226644  Topic10  -5.3380   2.0778

[660 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
605       1  0.072404   abnorm
605       2  0.144808   abnorm
605       4  0.217212   abnorm
605       7  0.144808   abnorm
605       8  0.362021   abnorm
...     ...       ...      ...
81        4  0.142212     work
81        5  0.331828     work
81        6  0.142212     work
81        8  0.142212     work
1669      4  0.763304  wrongli

[1306 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 9, 3, 7, 2, 5, 4, 10, 8])